In [1]:
%matplotlib inline
from os import listdir, path
from requests import get
import urllib3
from lxml import etree
import pandas as pd
import warnings
from tqdm import tqdm
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
#Data Rutte2
for year in tqdm(range(2011, 2018)):
    endOfAssembly = False
    assemblyNr = 0
    while endOfAssembly == False:
        assemblyNr += 1
        for debateNr in range(1, 200):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(assemblyNr), str(debateNr))
            if not path.isfile('HandelingenTK/%s.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    # Download the Handelingen file
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    # Download the corresponding meta file
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    
                elif debateNr == 1:
                    endOfAssembly = True
                    break
                else:
                    break

In [ ]:
#Data Balkenende
for year in tqdm(range(2006, 2011)):
    for i in range(1,10000):
        for j in range(i,i+100):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(i), str(j))
            if not path.isfile('HandelingenTK/%s.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    # Download the Handelingen file
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    # Download the corresponding meta file
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

In [ ]:
enddict = {}
index = 0
for file in listdir('HandelingenTK/'):
    context = etree.iterparse('HandelingenTK/' + file,  tag='spreekbeurt')
    mainSpeaker = None
    for _, elem in context:
        if elem.get('nieuw') == 'ja':
            speechCategory = "Main Speech"
            mainSpeaker = elem.findtext('spreker/naam/achternaam')
        elif elem.findtext('spreker/naam/achternaam') == mainSpeaker:
            speechCategory = "Response"
        else:
            speechCategory = "Interruption"
        surname = elem.findtext('spreker/naam/achternaam')
        party = elem.findtext('spreker/politiek')
        text = ' '.join(elem.find('tekst').itertext())
        enddict[index] = [surname, party, text, file, speechCategory]
        del surname, party, text, speechCategory
        index += 1
df = pd.DataFrame.from_dict(enddict, orient='index')
df = df.rename(columns={0:'surname', 1:'party', 2:'text', 3:'file', 4:'speech category'})
df.head(5)

In [ ]:
metaDict = {}
for file in listdir('HandelingenTKmeta'):
    tags = []
    date = None
    context = etree.iterparse('HandelingenTKmeta/' + file,  tag='metadata')
    for _, elem in context:
        if elem.get('name') == "OVERHEIDop.datumVergadering":
            date = elem.get('content')
        elif elem.get('name') == "OVERHEID.category":
            tags.append(elem.get('content'))
        metaDict[file[:-12] + '.xml'] = [date, tags]
metaDF = pd.DataFrame.from_dict(metaDict, orient='index')
metaDF = metaDF.rename(columns={0:'date', 1:'tags'})
metaDF.head(5)

In [ ]:
rutte2DF = df.join(metaDF, on='file')
rutte2DF = rutte2DF.dropna()
#https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates
rutte2DF['date'] = pd.to_datetime(rutte2DF['date'])
mask = (rutte2DF['date'] > '2012-11-05') & (rutte2DF['date'] <= '2017-03-23')
rutte2DF = rutte2DF.loc[mask]
rutte2DF = rutte2DF[rutte2DF['party'].isin(['50PLUS', 'CDA', 'ChristenUnie', 'D66', 'GroenLinks', 'PVV', 'PvdA', 'PvdD', 'SGP', 'SP', 'VVD'])]
rutte2DF.to_csv("HandelingenRutte2.csv")
rutte2DF.head(5)

In [ ]:
balkenende4DF = df.join(metaDF, on='file')
print(balkenende4DF.head(5))
balkenende4DF = balkenende4DF.dropna()
#https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates
balkenende4DF['date'] = pd.to_datetime(balkenende4DF['date'])
mask = (balkenende4DF['date'] > '2007-02-22') & (balkenende4DF['date'] <= '2010-02-20')
balkenende4DF = balkenende4DF.loc[mask]
#combineddf = combineddf[combineddf['party'].isin(['50PLUS', 'CDA', 'ChristenUnie', 'D66', 'GroenLinks', 'PVV', 'PvdA', 'PvdD', 'SGP', 'SP', 'VVD'])]
balkenende4DF.to_csv("HandelingenBalkenende4.csv")
balkenende4DF.head(5)